In [2]:
from CreateDocuments import load_documents
import RAG_utils

import pandas as pd
import tqdm
%load_ext autoreload
%autoreload 2

In [ ]:
db = RAG_utils.create_chroma_db(doc_path='documents/test_size_1000_overlap_200_documents.pkl', collection_name='test')

In [3]:
db = RAG_utils.access_chroma_db(collection_name='train_full')

In [4]:
df = pd.read_parquet('rag-dataset-12000/data/test-00000-of-00001-af2a9f454ad1b8a3.parquet')
df.head(2)

,context,question,answer
0,"HOUSTON (Jan. 23, 2018) – Fabien Gabel, music ...",Who is the music director of the Quebec Sympho...,The music director of the Quebec Symphony Orch...
1,Port Harcourt – The Rivers police command on W...,Who were the four students of the University o...,The four students of the University of Port Ha...


In [ ]:
# iterate rows
rows = []
for i, row in tqdm.tqdm(df.iterrows(), total=df.shape[0]):
    idx = i
    question = row['question']
    ground_truth_answer = row['answer']

    system_message = """You are a helpful assistant. Answer the user's question in one sentence based on the provided context. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Do NOT start your response with "According to the provided context." """
    user_message_template = """Context: {context} Question: {question}"""

    documents = db.similarity_search_with_relevance_scores(question, k=5)
    contexts = [doc[0].page_content for doc in documents]
    #context = RAG_utils.format_docs([doc[0] for doc in documents])

    user_message = user_message_template.format(context=contexts, question=question)

    answer = RAG_utils.gen_text_ollama(sys_msg=system_message, user_msg=user_message,options={'seed':0, 'temperature':0.01})

    row = [idx, question, answer, contexts, ground_truth_answer]
    rows.append(row)
    # eval_df = pd.concat([eval_df, pd.DataFrame([row])], ignore_index=True)
    if i > 2:
        break

eval_df = pd.DataFrame(rows, columns=['index','question', 'answer', 'contexts', 'ground_truth'])
eval_df
from datasets import Dataset
dataset = Dataset.from_pandas(eval_df)

In [ ]:
eval_df['contexts'][0]

In [ ]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, answer_relevancy, context_recall, context_entity_recall

from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

# langchain_llm = Ollama(model="llama3")
# langchain_embeddings = OllamaEmbeddings()

results = evaluate(dataset, metrics=[faithfulness, answer_correctness, context_recall, context_entity_recall, answer_relevancy], llm=Ollama(model="llama3"), embeddings=OllamaEmbeddings())
results.to_pandas()


In [ ]:
results.to_pandas().to_csv('RAGAS_test_eval_sample.csv')

In [ ]:
results.to_pandas().iloc[1]['ground_truth']

In [ ]:
# https://docs.ragas.io/en/latest/concepts/metrics/context_relevancy.html
from ragas.metrics import ContextRelevancy
context_relevancy = ContextRelevancy()


# Dataset({
#     features: ['question','contexts'],
#     num_rows: 25
# })
dataset: Dataset

results = context_relevancy.score(dataset)


In [ ]:
df.iloc[0]

In [ ]:
i = 0
question = df.iloc[i]['question']
ground_truth_answer = df.iloc[i]['answer']

system_message = """You are a helpful assistant. Answer the user's question in one sentence based on the provided context. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Do NOT start your response with "According to the provided context." """
user_message_template = """Context: {context} Question: {question}"""

documents = db.similarity_search_with_relevance_scores(question, k=5)
context = RAG_utils.format_docs([doc[0] for doc in documents])


In [ ]:

context = RAG_utils.format_docs([doc[0] for doc in documents])
user_message = user_message_template.format(context=context, question=question)

answer = RAG_utils.gen_text_ollama(sys_msg=system_message, user_msg=user_message,options={'seed':0, 'temperature':0.01})
print(question)
print(answer)

score = evaluator.evaluate(context=context, ground_truth_answer=ground_truth_answer, generated_response=answer)
score



In [6]:
question = 'When was the house at 3524 Redwing Ct, Naperville, IL 60564 last sold and for what price?'

# prompt_template1 = """Your are a helpful assistant. Please answer in one sentence. Answer the question based only on the following context:
# {context}
# Question: {question}
# Answer: 
# """

sys_msg = """You are a helpful assistant. Answer the user's question in one sentence based on the provided context. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Do NOT start your response with "According to the provided context." """
zephyr_prompt_template = """<system>{sys_msg}</s>\n<user>{user_msg}</s>\n<|assistant|>"""
user_msg_template = """Context: {context} Question: {question}"""


documents = db.similarity_search_with_relevance_scores(question, k=5)
context = RAG_utils.format_docs([doc[0] for doc in documents])

user_msg = user_msg_template.format(context=context, question=question)
prompt_text = zephyr_prompt_template.format(sys_msg=sys_msg, user_msg=user_msg)

answer = RAG_utils.gen_text_hf_api(lm_name='HuggingFaceH4/zephyr-7b-beta', prompt_text=prompt_text)
print(question)
print(answer)

When was the house at 3524 Redwing Ct, Naperville, IL 60564 last sold and for what price?

The house at 3524 Redwing Ct, Naperville, IL 60564 was last sold in October 2013 for $595,000, as provided in the given context.


In [7]:
qa_df = pd.read_parquet('rag-dataset-12000/data/train-00000-of-00001-9df3a936e1f63191.parquet')


# With Bedrock
https://docs.ragas.io/en/stable/howtos/customisations/aws-bedrock.html